## Assignment week 05: Sleeping habits

Welcome to **week five** of this course programming 1. You will learn about analysing data with pandas and numpy and you will learn to visualize with bokeh. Concretely, you will preprocess the Sleep Study data in an appropiate format in order to conduct statistical and visual analysis. Learning outcomes:


## About the data

The data is collected from a survey-based study of the sleeping habits of individuals within the US. 

Below is a description of each of the variables contained within the dataset.

- Enough = Do you think that you get enough sleep?
- Hours = On average, how many hours of sleep do you get on a weeknight?
- PhoneReach = Do you sleep with your phone within arms reach?
- PhoneTime = Do you use your phone within 30 minutes of falling asleep?
- Tired = On a scale from 1 to 5, how tired are you throughout the day? (1 being not tired, 5 being very tired)
- Breakfast = Do you typically eat breakfast?

The two research questions you should answer in this assignment are:
1. Is there a differences in Hours sleep caused by having breakfast (yes, no)?
2. Is there a differences in Hours sleep caused by having breakfast and the tireness (score)


The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: data inspection](#1)
- [part 3: check assumptions](#2)
   - [check normality 3.1](#ex-31)
   - [check equal variance 3.2](#ex-32)
- [part 4: prepare the data](#3)
- [part 5: answer the research question](#4)
- [part 6: enhanced plotting](#5)

Part 1 till 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


**NOTE If your project data is suitable you can use that data instead of the given data**

## ANOVA

Analysis of variance (ANOVA) compares the variances between groups versus within groups. It basically determines whether the differences between groups is larger than the differences within a group (the noise). 
A graph picturing this is as follow: https://link.springer.com/article/10.1007/s00424-019-02300-4/figures/2


In ANOVA, the dependent variable must be a continuous (interval or ratio) level of measurement. For instance Glucose level. The independent variables in ANOVA must be categorical (nominal or ordinal) variables. For instance trial category, time of day (AM versus PM) or time of trial (different categories). Like the t-test, ANOVA is also a parametric test and has some assumptions. ANOVA assumes that the data is normally distributed.  The ANOVA also assumes homogeneity of variance, which means that the variance among the groups should be approximately equal. ANOVA also assumes that the observations are independent of each other. 

A one-way ANOVA has just one independent variable. A two-way ANOVA (are also called factorial ANOVA) refers to an ANOVA using two independent variables. For research question 1 we can use the one-way ANOVA, for research question two we can use two-way ANOVA. But first we need to check the assumptions. 


---

In [124]:
# IMPORTS
import numpy as np
import pandas as pd
from pathlib import Path
import yaml
import hvplot.pandas
import holoviews as hv
from scipy.stats import shapiro

In [125]:
# IMPORTS
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot, column
from bokeh.plotting import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import CustomJS, Dropdown
from bokeh.models import Span
from bokeh.transform import jitter
import panel as pn
import regex as re

output_notebook()
pn.extension()

Loading BokehJS ...

In [126]:
def get_config():
    """
    Read in config file and return it as a dictionary.
    
    :returns
    --------
    config - dict
        Configuration file in dictionary form.
    """
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

config = get_config()
print(type(config))
data_dir = config['datadir']

<class 'dict'>


<a name='0'></a>
## Part 1: Load the data (10 pt)

load the `sleep.csv` data. Get yourself familiar with the data. Answer the following questions.

1. What is the percentage missing data?
2. Considering the research question, what is the dependent variable and what are the indepent variables? Are they of the correct datatype? 

In [127]:
file = "sleep.csv"
sleep_data = pd.read_csv(Path(data_dir + file))
sleep_data

,Enough,Hours,PhoneReach,PhoneTime,Tired,Breakfast
0,Yes,8,Yes,Yes,3,Yes
1,Yes,6,Yes,Yes,2,Yes
2,No,7,Yes,Yes,4,No
3,No,7,Yes,Yes,2,Yes
4,No,7,Yes,Yes,4,No
...,...,...,...,...,...,...
97,No,7,Yes,Yes,2,Yes
98,No,7,No,Yes,3,Yes
99,Yes,8,Yes,Yes,3,Yes
100,Yes,7,Yes,Yes,2,Yes


In [128]:
# Remove white spaces from column names
new_cols = sleep_data.columns.str.strip()
sleep_data.columns = new_cols

In [129]:
# Turn empty cells into NaN values
sleep_data = sleep_data.replace(r'^\s*$', np.NaN, regex=True)

#code printing percentage missing data
n_missing_values = sleep_data.isnull().sum().sum() # number missing values glucose column
n_total_values = sleep_data.shape[0] # number of total values
percentage_missing = n_missing_values / n_total_values
print(f"Percentage missing data: {percentage_missing}\n")

Percentage missing data: 0.0196078431372549



1. Is there a differences in Hours sleep caused by having breakfast (yes, no)?

In [130]:
#code printing answer dependent and independent variables
print("Research question one:\n")
print("Dependent variable = Hours sleep")
print("Independent variable = Eaten breakfast (yes/no) ")

Research question one:

Dependent variable = Hours sleep
Independent variable = Eaten breakfast (yes/no) 


2. Is there a differences in Hours sleep caused by having breakfast and the tireness (score)

In [131]:
print("Research question two:\n")
print("Dependent variable = Hours sleep")
print("Independent variable = Eaten breakfast (yes/no) and tiredness ")

Research question two:

Dependent variable = Hours sleep
Independent variable = Eaten breakfast (yes/no) and tiredness 


In [132]:
#code printing answer about datatypes

print("""The variables are not of the correct data types. The data type of the'Hours' variable needs to be changed to int or 
float. And the data type of the independent variables 'Breakfast' and 'Tired' need to be set to catergory.\n""")

# Turn 'Hours' data type into float64
sleep_data['Hours'] = pd.to_numeric(sleep_data['Hours'], errors = 'coerce').astype("float64")

sleep_data.info()

The variables are not of the correct data types. The data type of the'Hours' variable needs to be changed to int or 
float. And the data type of the independent variables 'Breakfast' and 'Tired' need to be set to catergory.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Enough      102 non-null    object 
 1   Hours       100 non-null    float64
 2   PhoneReach  102 non-null    object 
 3   PhoneTime   102 non-null    object 
 4   Tired       102 non-null    int64  
 5   Breakfast   102 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.9+ KB


---

<a name='1'></a>
## Part 2: Inspect the data (30 pt)

Inspect the data practically. Get an idea about how well the variable categories are ballanced. Are the values of a variable equally divided? What is the mean value of the dependent variable? Are there correlations amongs the variables?


<ul>
<li>Create some meaninful overviews such as variable value counts</li>
<li>Create a scatter plot ploting the relation between being tired and hours of sleep with different colors for Breakfast</li>
    <li>Print some basic statistics about the target (mean, standard deviation)</li>
    <li>Create a heatmap to check for correlations among variables. </li>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>the gitbook has a bokeh heatmap example</li></ul>
</details>
</ul>

In [133]:
#code your answer to the value counts and distribution plots here
print("Value counts for the dependent variable:")
val_counts_hours = sleep_data["Hours"].value_counts()
print(val_counts_hours)

mean = sleep_data["Hours"].mean()
print(f"Mean value of the dependent variable ('Hours'): {mean}\n")

print("Value counts for the independent variable 'Tired':")
val_counts_tired = sleep_data["Tired"].value_counts()
print(f"{val_counts_tired}\n")

print("Value counts for the independent variable 'Breakfast':")
val_counts_breakfast = sleep_data["Breakfast"].value_counts()
print(val_counts_breakfast)

# Plot distribution Hours slept and breakfast
sleep_data[['Hours', 'Breakfast']].hvplot.kde(by='Breakfast', legend = 'top_right', color = ['red', 'green'])

Value counts for the dependent variable:
7.0     34
6.0     23
8.0     16
5.0     12
9.0      8
4.0      4
2.0      2
10.0     1
Name: Hours, dtype: int64
Mean value of the dependent variable ('Hours'): 6.66

Value counts for the independent variable 'Tired':
3    38
2    27
4    23
5    10
1     4
Name: Tired, dtype: int64

Value counts for the independent variable 'Breakfast':
Yes    62
No     40
Name: Breakfast, dtype: int64


:NdOverlay   [Breakfast]
   :Distribution   [Hours]   (Density)

In [134]:
#code for the scatter plot here

# Create two seperate data frames for yes and no breakfast
yes_breakfast = sleep_data[sleep_data['Breakfast'] == 'Yes'][['Hours', 'Tired']]
no_breakfast = sleep_data[sleep_data['Breakfast'] == 'No'][['Hours', 'Tired']]

# Create ColumnDataSource objects
source_yes = ColumnDataSource(yes_breakfast)
source_no = ColumnDataSource(no_breakfast)

p = figure(title = "Relation between being tired and hours slept.",plot_width = 750, plot_height = 400, tools="pan, hover, zoom_in, zoom_out, yzoom_in, yzoom_out")


# Create the points for patient who have died
points = p.scatter(jitter('Tired', 0.5), 'Hours', source=source_yes, color = "green", marker = "dot", size = 30, legend_label = "Breakfast: yes", alpha = 0.7)
# Create the points for patient who survived
points2 = p.scatter(jitter('Tired', 0.5), 'Hours', source=source_no, color = "red", marker = "dot", size = 30, legend_label = "Breakfast: no", alpha = 0.5)

# Set labels
p.xaxis.axis_label = 'Tirednes (1 being not tired, 5 being very tired))'
# Use regex to grab the info about what was measured
p.yaxis.axis_label = 'Hours slept'

# Make legend interactive
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p)

In [135]:
#code your answer to the target statistics here
print(f"Descriptive statistics 'Hours': mean = {sleep_data['Hours'].mean()}, sd = {sleep_data['Hours'].std()}, variance = {sleep_data['Hours'].var()}")

Descriptive statistics 'Hours': mean = 6.66, sd = 1.4299819875958169, variance = 2.0448484848484827


In [136]:
#code your answer for the heatmap here and briefly state your finding

# sleep_data["Enough"].str.strip().map({"Yes": 1, "No":0})

sleep_data_num = sleep_data.copy()

# Yes/no columns
cols = ["Enough", "PhoneReach", "PhoneTime", "Breakfast"]
# Turn yes/no values to 1 and 0.
sleep_data_num[cols] = sleep_data_num[cols].applymap(lambda x: 1 if x.strip()=="Yes" else 0)

c = sleep_data_num.corr().abs()
y_range = (list(reversed(c.columns)))
x_range = (list(c.index))
# c
c

,Enough,Hours,PhoneReach,PhoneTime,Tired,Breakfast
Enough,1.000000,0.379229,0.089882,0.007139,0.421483,0.115278
Hours,0.379229,1.000000,0.053796,0.153000,0.192382,0.220936
PhoneReach,0.089882,0.053796,1.000000,0.145044,0.072554,0.240141
PhoneTime,0.007139,0.153000,0.145044,1.000000,0.034775,0.007934
Tired,0.421483,0.192382,0.072554,0.034775,1.000000,0.254123
Breakfast,0.115278,0.220936,0.240141,0.007934,0.254123,1.000000


In [137]:
#reshape
dfc = pd.DataFrame(c.stack(), columns=['r']).reset_index()
dfc.head()

,level_0,level_1,r
0,Enough,Enough,1.000000
1,Enough,Hours,0.379229
2,Enough,PhoneReach,0.089882
3,Enough,PhoneTime,0.007139
4,Enough,Tired,0.421483


In [138]:
#plot a heatmap
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource,
                          LinearColorMapper, PrintfTickFormatter,)
from bokeh.transform import transform
from bokeh.palettes import Viridis256

source = ColumnDataSource(dfc)

#create colormapper 
mapper = LinearColorMapper(palette=Viridis256, low=dfc.r.min(), high=dfc.r.max())

#create plot
p = figure(title="correlation heatmap", plot_width=500, plot_height=450,
           x_range=x_range, y_range=y_range, x_axis_location="above", toolbar_location=None)

#use mapper to fill the rectangles in the plot
p.rect(x="level_0", y="level_1", width=1, height=1, source=source,
       line_color=None, fill_color=transform('r', mapper))

#create and add colorbar to the right
color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(x_range)), 
                     formatter=PrintfTickFormatter(format="%.1f"))
p.add_layout(color_bar, 'right')

#draw axis
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0

#show
show(p)

From the heatmap we can't determine a clear relation between two variables. This is probably because it is mostly categorical data.

---

<a name='2'></a>
## Part 3: Check Assumptions

Before we answer the research question with ANOVA we need to check the following assumptions:

1. ANOVA assumes that the dependent variable is normaly distributed
2. ANOVA also assumes homogeneity of variance
3. ANOVA also assumes that the observations are independent of each other. Most of the time we need domain knowledge and experiment setup descriptions to estimate this assumption

We are going to do this graphically and statistically. 

<a name='ex-31'></a>
### Check normality (10 pt)

<ul><li>
Plot the distribution of the dependent variable. Add a vertical line at the position of the average. Add a vertical line for the robuust estimation. Add the normal distribution line to the plot. Comment on the normallity of the data. Do you want the full points? Plot with bokeh!</li>

<li>Use a Shapiro-Wilk Test or an Anderson-Darling test to check statistically</li></ul>


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>check the code of lesson 1 DS1 bayesian statistics</li>
        <li>heart_failure case of gitbook uses bokeh histograms</li>
</ul>
</details>

In [139]:
from scipy.stats import iqr # iqr is the Interquartile Range function
from scipy.stats import norm

In [140]:
def Q_Q_Plot(y, est = 'robust', **kwargs):
    
    ################################################################################
    #
    # Arguments:
    #
    # y                  data array
    # est                Estimation method for normal parameters mu and sigma:
    #                    either 'robust' (default), or 'ML' (Maximum Likelihood),
    #                    or 'preset' (given values)
    # If est='preset' than the optional parameters mu, sigma must be provided
    #
    # Author:            M.E.F. Apol
    # Date:              2020-01-06
    #
    ################################################################################
    
    import numpy as np
    from scipy.stats import iqr # iqr is the Interquartile Range function
    import matplotlib.pyplot as plt
    
    # First, get the optional arguments mu and sigma:
    mu_0 = kwargs.get('mu', None)
    sigma_0 = kwargs.get('sigma', None)
    
    n = len(y)
    
    # Calculate order statistic:
    y_os = np.sort(y)
  
    # Estimates of mu and sigma:
    # ML estimates:
    mu_ML = y.mean()
    sigma2_ML = np.mean((y.values - 6.66)**2, where=~np.isnan(y))
    
    sigma_ML = np.sqrt(sigma2_ML) # biased estimate
    s2 = n/(n-1) * sigma2_ML
    s = np.sqrt(s2) # unbiased estimate
    # Robust estimates:
    mu_R = y.median()
    sigma_R = iqr(y, nan_policy = 'omit')/1.349

    # Assign values of mu and sigma for z-transform:
    if est == 'ML':
        mu, sigma = mu_ML, s
    elif est == 'robust':
        mu, sigma = mu_R, sigma_R
    elif est == 'preset':
        mu, sigma = mu_0, sigma_0
    else:
        print('Wrong estimation method chosen!')
        
    print('Estimation method: ' + est)
    print('mu = ',mu,', sigma = ',sigma)
        
    # Perform z-transform: sample quantiles z.i
    z_i = (y_os - mu)/sigma

    # Calculate cumulative probabilities p.i:
    i = np.array(range(n)) + 1
    p_i = (i - 0.5)/n

    # Calculate theoretical quantiles z.(i):
    from scipy.stats import norm
    z_th = norm.ppf(p_i, 0, 1)

    # Calculate SE or theoretical quantiles:
    SE_z_th = (1/norm.pdf(z_th, 0, 1)) * np.sqrt((p_i * (1 - p_i)) / n)

    # Calculate 95% CI of diagonal line:
    CI_upper = z_th + 1.96 * SE_z_th
    CI_lower = z_th - 1.96 * SE_z_th

    return z_th, z_i, CI_lower, CI_upper

In [141]:
# Get Q-Q plot data
y = sleep_data.Hours

z_th, z_i, CI_lower, CI_upper = Q_Q_Plot(y, est = 'ML')

Estimation method: ML
mu =  6.66 , sigma =  1.4298403982110857


### Q-Q plot

In [142]:
p = figure(title = "Q-Q plot: check normality hours slept.",plot_width = 750, plot_height = 400, tools="pan, hover, zoom_in, zoom_out, yzoom_in, yzoom_out")


# Create the points of the experimental daata
points = p.scatter(z_th, z_i, color = "black", marker = "dot", size = 30, legend_label = "experimental data", alpha = 0.7)
line_normal = p.line(z_th, z_th, color = "red", legend_label = "normal line", line_dash = "dashdot")
# Create the lines for the 95% CI interval
line_lower = p.line(z_th, CI_lower, color = "blue", legend_label = "95% CI", line_dash = "dashdot")
line_upper = p.line(z_th, CI_upper, color = "blue", legend_label = "95% CI", line_dash = "dashdot")

# Set labels
p.xaxis.axis_label = 'Theoretical quantiles, Z(i)'
p.yaxis.axis_label = 'Sample quantiles, Zi'


# Make legend interactive
p.legend.location = "top_left"
p.legend.click_policy="hide"

show(p)

### Histogram

In [143]:
y = sleep_data.Hours[~np.isnan(sleep_data.Hours)]

n = len(y)

hist, edges = np.histogram(y, density = True)

# ML estimation
mu_ML = np.mean(y)
sigma2_ML = np.mean((y - mu_ML)**2)
sigma_ML = np.sqrt(sigma2_ML) # biased estimate
s2 = n/(n-1) * sigma2_ML
s = np.sqrt(s2) # unbiased estimate

# Robust estimates:
mu_R = np.median(y)
sigma_R = iqr(y)/1.349

# Calculate the CLT normal distribution:
x = np.linspace(np.min(y), np.max(y), 501)
rv_ML = np.array([norm.pdf(xi, loc = mu_ML, scale = s) for xi in x])

rv_R = np.array([norm.pdf(xi, loc = mu_R, scale = sigma_R) for xi in x])

p = figure()
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5, legend_label = "Hours slept")
p.line(x, rv_ML, color = "red", line_dash = "dashdot", legend_label = "ML estimation")
vline_ML = Span(location=mu_ML, dimension='height', line_color='red', line_width=1, line_dash = "dashdot")

p.line(x, rv_R, color = "blue", line_dash = "dashdot", legend_label = "Robust estimation")
vline_R = Span(location=mu_R, dimension='height', line_color='blue', line_width=1, line_dash = "dashdot")

p.add_layout(vline_ML)
p.add_layout(vline_R)
show(p)

In [144]:
# briefly summarize your findings
print("""According to the Q-Q plot the dependent variable has a normal distribution, if we use a robust estimation. However,
there are a few points which fall out of the 95% CI bounds. If we look at the histogram we can see both the robust and the 
maximum likelihood estimation.""")

t_stat, p_val = shapiro(y)
print(f"\nHowever, if we use the Shapiro-Wilk test to check normality, we find that the data does not follow a normal " + 
f"distribution. From the test we get a p-value of: {p_val} which is below the alpha (p-value < 0.05). That means that we " + 
f"cannot reject the null hypothesis which states that the data is NOT normally distributed.")

According to the Q-Q plot the dependent variable has a normal distribution, if we use a robust estimation. However,
there are a few points which fall out of the 95% CI bounds. If we look at the histogram we can see both the robust and the 
maximum likelihood estimation.

However, if we use the Shapiro-Wilk test to check normality, we find that the data does not follow a normal distribution. From the test we get a p-value of: 9.799786494113505e-05 which is below the alpha (p-value < 0.05). That means that we cannot reject the null hypothesis which states that the data is NOT normally distributed.


<a name='ex-32'></a>
### Check homogeneity of variance (20 pt)

<ul><li>
Use boxplots for the check of homoegeneity of variance. Do you want the full points? Plot with bokeh!</li>

<li>Use a Levene’s & Bartlett’s Test of Equality (Homogeneity) of Variance to test equal variance statistically</li><ul>

In [145]:
import hvplot.pandas
import holoviews as hv

In [155]:
# your code to plot here

boxplot = sleep_data.hvplot.box(y="Hours", by = ['Tired', 'Breakfast'],  color = "Breakfast",cmap = ['red', 'green'])
boxplot

:BoxWhisker   [Tired,Breakfast]   (Hours)

In [148]:
# your code for the statistical test here

In [149]:
# briefly summarize your findings

---

<a name='3'></a>
## Part 4: Prepare your data (10 pt)

Create a dataframe with equal samplesize. Make three categories for tireness 1-2 = no, 3 = maybe, 4-5 = yes

In [150]:
#your solution here

# Set dtype to category
# sleep_data['Breakfast'] = sleep_data['Breakfast'].astype("category")

# Set dtype to category
# sleep_data['Tired'] = sleep_data['Tired'].astype("category")

# sleep_data['Tired'] = sleep_data['Tired'].map({1: "died", 0:"alive"})

---

<a name='4'></a>
## Part 5: Answer the research questions (20 pt)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use one-way ANOVA for research question 1</li>
    <li>Use two-way ANOVA for research question 2</li>
    <li>https://reneshbedre.github.io/blog/anova.html</li>
</ul>
</details>

In [151]:
#Your solution here

---

<a name='5'></a>
## Part 6: Enhanced plotting (20 pt)

Create a panel with 1) your dataframe with equal samplesize 2) a picture of a sleeping beauty, 3) the scatter plot of tired / hours of sleep with different colors for Breakfast from part 2 4) the boxplots given the p-value for the anova outcome in the title

In [152]:
#your solution here